In [ ]:
# import models
from sleep_classif.CNNmultitaper import ConvNetMultitaper
from sleep_classif.LSTMConv import LSTM_Conv
from sleep_classif.CNNadvanced import CNN_Advanced
from sleep_classif.CNNmodel import SimpleCNN

# import loaders and other functions
from sleep_classif.preprocessing import compute_tapers
from sleep_classif.dataloaders import MultiTaperSet, RawDataSet, FFT_Raw_DataSet
from sleep_classif.trainer import Trainer

# import from other librairies 
import torch
import torch.nn as nn
import random
import numpy as np

## Prepare Cuda


In [ ]:
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'
print(device)

## K_fold indices generator

In [ ]:
def k_fold_indices(set_size, n_folds = 5):
    '''
    return list of folds indices (train indices and  test indices)
    '''
    s = list(range(0, set_size))
    random.shuffle(s)
    s = [s[i::n_folds] for i in range(n_folds)]
    folds = []
    for i in range(n_folds):
        test_set = np.array(s[i])
        train_set = np.array([s[j] for j in range(n_folds) if i!=j]).ravel()
        folds.append((train_set, test_set))
    return(folds)

In [ ]:
k_fold_indices(10, n_folds = 5)

## Train basic CNN network

In [ ]:
data_path_train = './data/raw_data/X_train.h5'
target_path = './data/raw_data/y_train.csv'

raw_train_set = RawDataSet(device=device,
                                 data_path = data_path_train,
                                 target_path = target_path)

In [ ]:
### train with K_fold 

acc_fold_list = []
n_folds = 5
n_epochs = 30
batch_size = 64

for n_fold, (train_indices, val_indices) in enumerate(k_fold_indices(len(raw_train_set), n_folds = n_folds)):
        print(f"fold number: {n_fold+1}")
        #instanciate a new model 
        simple_cnn = SimpleCNN().to(device)
        optimizer = torch.optim.Adam(simple_cnn.parameters())

        # creata data sets
        train_set = torch.utils.data.dataset.Subset(raw_train_set,train_indices)
        val_set = torch.utils.data.dataset.Subset(raw_train_set,val_indices)

        train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
        val_loader = torch.utils.data.DataLoader(val_set, batch_size=batch_size, shuffle=True)

        # instanciate trainer
        trainer = Trainer(simple_cnn,
                        nn.CrossEntropyLoss(),
                        optimizer,
                        train_loader,
                        device,
                        valid_data_loader = val_loader,
                        class_weights=torch.Tensor([8.081897,22.222222, 2.756846, 3.765060, 4.927727]))

        # train model
        loss_list = []
        accuracy_list = []
        for epoch in range(0,n_epochs):
                loss, accuracy = trainer.train_epoch()
                if epoch == n_epochs-1:
                        acc_fold_list.append(accuracy)

# return mean accuracy
accuracy = np.mean(acc_fold_list)
print(f"K_fold average accuracy: {accuracy}")

## Train CNN + Multitaper Model

### Create MultiTapers

In [ ]:
compute_tapers()

In [ ]:
features_eeg_path_train = './data/pre_processed_data/Multitaper_eeg_train.npy'
features_position_path_train = './data/pre_processed_data/Multitaper_position_train.npy'

target_path = './data/raw_data/y_train.csv'

taper_train_set = MultiTaperSet(device=device,
                                features_eeg_path = features_eeg_path_train,
                                features_position_path = features_position_path_train,
                                target_path = target_path)

In [ ]:
### train with K_fold 

acc_fold_list = []
n_folds = 5
n_epochs = 30
batch_size = 64

for n_fold, (train_indices, val_indices) in enumerate(k_fold_indices(len(taper_train_set), n_folds = n_folds)):
        print(f"fold number: {n_fold+1}")
        #instanciate a new model 
        CNN_taper_model = ConvNetMultitaper().to(device)
        optimizer = torch.optim.Adam(CNN_taper_model.parameters())

        # creata data sets
        train_set = torch.utils.data.dataset.Subset(taper_train_set,train_indices)
        val_set = torch.utils.data.dataset.Subset(taper_train_set,val_indices)

        train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
        val_loader = torch.utils.data.DataLoader(val_set, batch_size=batch_size, shuffle=True)

        # instanciate trainer
        trainer = Trainer(CNN_taper_model,
                        nn.CrossEntropyLoss(),
                        optimizer,
                        train_loader,
                        device,
                        valid_data_loader = val_loader,
                        class_weights=torch.Tensor([8.081897,22.222222, 2.756846, 3.765060, 4.927727]))

        # train model
        loss_list = []
        accuracy_list = []
        for epoch in range(0,n_epochs):
                loss, accuracy = trainer.train_epoch()
                if epoch == n_epochs-1:
                        acc_fold_list.append(accuracy)

# return mean accuracy
accuracy = np.mean(acc_fold_list)
print(f"K_fold average accuracy: {accuracy}")

## Training an advanced CNN network

In [ ]:
from scipy import fftpack

data_path_train = './data/raw_data/X_train.h5'
target_path = './data/raw_data/y_train.csv'



raw_train_set = FFT_Raw_DataSet(device=device,
                                 data_path = data_path_train,
                                 target_path = target_path)

In [ ]:
### train with K_fold 

acc_fold_list = []
n_folds = 5
n_epochs = 30
batch_size = 64

num_classes = 5
raw_feat, fft_feat, raw_pos_feat, fft_pos_feat = 5,5,3,3

for n_fold, (train_indices, val_indices) in enumerate(k_fold_indices(len(raw_train_set), n_folds = n_folds)):
        print(f"fold number: {n_fold+1}")
        #instanciate a new model 
        CNN_Advanced_model = CNN_Advanced(raw_feat, fft_feat, raw_pos_feat, fft_pos_feat, num_classes, 0.5).to(device)
        optimizer = torch.optim.Adam(CNN_Advanced_model.parameters())

        # creata data sets
        train_set = torch.utils.data.dataset.Subset(raw_train_set,train_indices)
        val_set = torch.utils.data.dataset.Subset(raw_train_set,val_indices)

        train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
        val_loader = torch.utils.data.DataLoader(val_set, batch_size=batch_size, shuffle=True)

        # instanciate trainer
        trainer = Trainer(CNN_Advanced_model,
                        nn.CrossEntropyLoss(),
                        optimizer,
                        train_loader,
                        device,
                        valid_data_loader = val_loader,
                        class_weights=torch.Tensor([8.081897,22.222222, 2.756846, 3.765060, 4.927727]))

        # train model
        loss_list = []
        accuracy_list = []
        for epoch in range(0,n_epochs):
                loss, accuracy = trainer.train_epoch()
                if epoch == n_epochs-1:
                        acc_fold_list.append(accuracy)

# return mean accuracy
accuracy = np.mean(acc_fold_list)
print(f"K_fold average accuracy: {accuracy}")

## Train CNN + LSTM

In [ ]:
data_path_train = './data/raw_data/X_train.h5'

target_path = './data/raw_data/y_train.csv'



raw_train_set = RawDataSet(device=device,
                                 data_path = data_path_train,
                                 target_path = target_path)

In [ ]:
### train with K_fold 

acc_fold_list = []
n_folds = 5
n_epochs = 30
batch_size = 64

raw_feat = raw_train_set.feature_shape()
num_classes = 5

for n_fold, (train_indices, val_indices) in enumerate(k_fold_indices(len(raw_train_set), n_folds = n_folds)):
        print(f"fold number: {n_fold+1}")
        #instanciate a new model 
        LSTM_Conv_model = LSTM_Conv(raw_feat, num_classes).to(device)
        optimizer = torch.optim.Adam(LSTM_Conv_model.parameters())

        # creata data sets
        train_set = torch.utils.data.dataset.Subset(raw_train_set,train_indices)
        val_set = torch.utils.data.dataset.Subset(raw_train_set,val_indices)

        train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
        val_loader = torch.utils.data.DataLoader(val_set, batch_size=batch_size, shuffle=True)

        # instanciate trainer
        trainer = Trainer(LSTM_Conv_model,
                        nn.CrossEntropyLoss(),
                        optimizer,
                        train_loader,
                        device,
                        valid_data_loader = val_loader,
                        class_weights=torch.Tensor([8.081897,22.222222, 2.756846, 3.765060, 4.927727]))

        # train model
        loss_list = []
        accuracy_list = []
        for epoch in range(0,n_epochs):
                loss, accuracy = trainer.train_epoch()
                if epoch == n_epochs-1:
                        acc_fold_list.append(accuracy)

# return mean accuracy
accuracy = np.mean(acc_fold_list)
print(f"K_fold average accuracy: {accuracy}")